In [ ]:
# Retrieval Augmented Generation (RAG) using LangChain
#%pip install langchain
#%pip install langchain-community
#%pip install sentence-transformers
#%pip install faiss-cpu
#%pip install bs4
#%pip install langchain-groq

####  LangChain Q&A Retriever
* ConversationalRetrievalChain
* Query the Source documents

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Step 1: Load the document from a web url
loader = WebBaseLoader(["https://huggingface.co/blog/llama31"])
documents = loader.load()

# Step 2: Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(documents)

# Step 3: Store the document into a vector store with a specific embedding model
vectorstore = FAISS.from_documents(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

First sign in at [Groq](https://console.groq.com/login) with your github or gmail account, then get an API token to try Groq out for free.

In [ ]:
import os
from getpass import getpass

GROQ_API_TOKEN = getpass(prompt="Enter GROQ API TOKEN: ")
os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

In [ ]:
from langchain.chains import ConversationalRetrievalChain

# Query against your own data
chain = ConversationalRetrievalChain.from_llm(llm,vectorstore.as_retriever(),return_source_documents=True)

# no chat history passed
result = chain({"question": "What’s new with Llama 3?", "chat_history": []})
utilsLlama.md(result['answer'])


In [ ]:
# This time your previous question and answer will be included as a chat history which will enable the ability
# to ask follow up questions.
query = "What two sizes?"
chat_history = [(query, result["answer"])]
result = chain({"question": query, "chat_history": chat_history})
utilsLlama.md(result['answer'])